# Other widget libraries

We would have loved to show you everything the Jupyter Widgets ecosystem has to offer today, but we are blessed to have such an active community of widget creators and unfortunately can't fit all widgets in a single session, no matter how long. 

This notebook lists some of the widget libraries we wanted to demo but did not have enough time to include in the session. Enjoy!

# ipyleaflet: Interactive maps

## A Jupyter - LeafletJS bridge

## https://github.com/jupyter-widgets/ipyleaflet


ipyleaflet is a jupyter interactive widget library which provides interactive maps to the Jupyter notebook.

- MIT Licensed

**Installation:**

```bash
conda install -c conda-forge ipyleaflet
```

In [ ]:
from ipywidgets import Text, HTML, HBox
from ipyleaflet import GeoJSON, WidgetControl, Map 
import json

In [ ]:
m = Map(center = (43,-100), zoom = 4)

geo_json_data = json.load(open('us-states-density-colored.json'))
geojson = GeoJSON(data=geo_json_data, hover_style={'color': 'black', 'dashArray': '5, 5', 'weight': 2})
m.add_layer(geojson)

html = HTML('''
    <h4>US population density</h4>
    Hover over a state
''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
m.add_control(control)

def update_html(properties, **kwargs):
    html.value = '''
        <h4>US population density</h4>
        <h2><b>{}</b></h2>
        {} people / mi^2
    '''.format(properties['name'], properties['density'])

geojson.on_hover(update_html)

m

# pythreejs: 3D rendering in the browser 

## A Jupyter - threejs bridge

## https://github.com/jupyter-widgets/pythreejs


Pythreejs is a jupyter interactive widget bringing fast WebGL 3d visualization to the Jupyter notebook.

- Originally authored by Jason Grout, currently maintained by Vidar Tonaas Fauske
- BSD Licensed

Pythreejs is *not* a 3d plotting library, it only exposes the threejs scene objects to the Jupyter kernel.

**Installation:**

```bash
conda install -c conda-forge pythreejs
```

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink

In [ ]:
# Generate surface data:
view_width = 600
view_height = 400
nx, ny = (20, 20)
xmax=1
x = np.linspace(-xmax, xmax, nx)
y = np.linspace(-xmax, xmax, ny)
xx, yy = np.meshgrid(x, y)
z = xx ** 2 - yy ** 2
#z[6,1] = float('nan')


# Generate scene objects from data:
surf_g = SurfaceGeometry(z=list(z[::-1].flat), 
                         width=2 * xmax,
                         height=2 * xmax,
                         width_segments=nx - 1,
                         height_segments=ny - 1)

surf = Mesh(geometry=surf_g,
            material=MeshLambertMaterial(map=height_texture(z[::-1], 'YlGnBu_r')))

surfgrid = SurfaceGrid(geometry=surf_g, material=LineBasicMaterial(color='black'),
                       position=[0, 0, 1e-2])  # Avoid overlap by lifting grid slightly

# Set up picking bojects:
hover_point = Mesh(geometry=SphereGeometry(radius=0.05),
                   material=MeshLambertMaterial(color='green'))

click_picker = Picker(controlling=surf, event='dblclick')
hover_picker = Picker(controlling=surf, event='mousemove')

# Set up scene:
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.4)
c = PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], aspect=view_width / view_height,
                      children=[key_light])

scene = Scene(children=[surf, c, surfgrid, hover_point, AmbientLight(intensity=0.8)])

renderer = Renderer(camera=c, scene=scene,
                    width=view_width, height=view_height,
                    controls=[OrbitControls(controlling=c), click_picker, hover_picker])


# Set up picking responses:
# Add a new marker when double-clicking:
out = Output()
def f(change):
    value = change['new']
    with out:
        print('Clicked on %s' % (value,))
    point = Mesh(geometry=SphereGeometry(radius=0.05), 
                 material=MeshLambertMaterial(color='hotpink'),
                 position=value)
    scene.add(point)

click_picker.observe(f, names=['point'])

# Have marker follow picker point:
link((hover_point, 'position'), (hover_picker, 'point'))

# Show picker point coordinates as a label:
h = HTML()
def g(change):
    h.value = 'Green point at (%.3f, %.3f, %.3f)' % tuple(change['new'])
    h.value += '  Double-click to add marker'
g({'new': hover_point.position})
hover_picker.observe(g, names=['point'])

display(VBox([h, renderer, out]))

# bqplot: complex interactive visualizations

## https://github.com/bloomberg/bqplot

## A Jupyter - d3.js bridge

bqplot is a jupyter interactive widget library bringing d3.js visualization to the Jupyter notebook.

- Apache Licensed

bqplot implements the abstractions of Wilkinson’s “The Grammar of Graphics” as interactive Jupyter widgets.

bqplot provides both
-	high-level plotting procedures with relevant defaults for common chart types,
-	lower-level descriptions of data visualizations meant for complex interactive visualization dashboards and applications involving mouse interactions and user-provided Python callbacks.

**Installation:**

```bash
conda install -c conda-forge bqplot
```

In [ ]:
import numpy as np
import bqplot as bq

In [ ]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x = np.arange(100)
y = np.cumsum(np.random.randn(2, 100), axis=1) #two random walks

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, colors=['red', 'green'])
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

fig = bq.Figure(marks=[line], axes=[xax, yax], animation_duration=1000)
display(fig)

In [ ]:
# update data of the line mark
line.y = np.cumsum(np.random.randn(2, 100), axis=1)

# ipympl: The Matplotlib Jupyter Widget Backend

## https://matplotlib.org/ipympl/


Enabling interaction with matplotlib charts in the Jupyter notebook and JupyterLab

- BSD-3-Clause

**Installation:**

```bash
conda install -c conda-forge ipympl
```

Enabling the `ipympl` (sometimes written `widget`) backend. ipympl can be install via pip or conda.

In [ ]:
%matplotlib ipympl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import VBox, FloatSlider

When using the `widget` backend from ipympl, `fig.canvas` is a proper Jupyter interactive widget, which can be embedded in Layout classes like `HBox` and `VBox`.

One can bind figure attributes to other widget values.

In [ ]:
with plt.ioff():
    fig, ax = plt.subplots()

x1 = np.linspace(0, 20, 500)

slider = FloatSlider(
    value=1.0,
    min=0.02,
    max=2.0
)

lines = plt.plot(x1, np.sin(slider.value  * x1))

def update_lines(change):
    lines[0].set_data(x1, np.sin(change.new * x1))
    fig.canvas.draw()

slider.observe(update_lines, names='value')

VBox([slider, fig.canvas])

# ipytree: Interactive tree view based on ipywidgets

## https://github.com/QuantStack/ipytree/


ipytree is a jupyter interactive widget library which provides a tree widget to the Jupyter notebook.

- MIT Licensed

**Installation:**

```bash
conda install -c conda-forge ipytree
```

## Create a tree

In [ ]:
from ipywidgets import Text, link
from ipytree import Tree, Node

In [ ]:
tree = Tree()
tree.add_node(Node('node1'))

node2 = Node('node2')
tree.add_node(node2)

tree

In [ ]:
node3 = Node('node3', disabled=True)
node4 = Node('node4')
node5 = Node('node5', [Node('1'), Node('2')])
node2.add_node(node3)
node2.add_node(node4)
node2.add_node(node5)